In [1]:
from PIL import Image, ImageFile        # Импорт Pillow для работы с изображениями
ImageFile.LOAD_TRUNCATED_IMAGES = True  # Обрезка изображений, которые не могут быть загружены полностью

import torch                   # Импорт PyTorch для работы с нейронными сетями
import torch.nn as nn          # Импорт модуля nn для настройки нейронных сетей
import torch.optim as optim    # Импорт optim для оптимизации работы
import torchvision             # Импорт TorchVision для компьютерного зрения
import torchvision.transforms as transforms  # Импорт модуля transforms для трансформации изображений

# Преобразование изображений для нормализации и изменения размера
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

'''
    Класс для обработки изображений через Pillow

    Метод getitem возвращает преобразованное изображение
'''
class ImageFolderWithPIL(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        image = Image.open(path).convert("RGB")
        if self.transform is not None:
            image = self.transform(image)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return image, target

# Создание набора изображений для тренировочного датасета
train_dataset = ImageFolderWithPIL(root="E:\Programming\RSVProject\dataSet\\train", transform=transform)
# Создание набора изображений для тестового датасета
test_dataset = ImageFolderWithPIL(root="E:\Programming\RSVProject\dataSet\\test", transform=transform)

# Создание загрузчиков данных
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

'''
    Реализация пользовательского класса Net, наследуемого от nn.Module из PyTorch.
    В классе описывается архитектура нейронной сети, 
    которая используется для классификации изображений
'''
class Net(nn.Module):
    '''
        Метод init описывает все слои нейронной сети с параметрами:
            conv1 первый слой на 3 входных канала, 16 выходных. Ядро 3х3
            relu определение функции активации ReLU
            pool определение слоя подвыборки стандартного размера
            conv2 второй слой - 16 входных каналов, 32 выходных. 
            fc1, fc2 линейное преобразование входных данных
        
        Метод forward функция прямого прохода, где x - входные данные

    '''
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 3)
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))   # Применение свёрточного слоя, ReLU и слоя подвыборки к данным
        x = self.pool(self.relu(self.conv2(x)))   # Повторное применение
        x = x.view(-1, 32 * 16 * 16)              # Приведение к форме первого полносвязного слоя
        x = self.relu(self.fc1(x))                # Применение ReLU к выходу первого полносвязного слоя
        x = self.fc2(x)                           # Применение второго полносвязного слоя к данным
        return x                                  # Возврат выходных данных сети

# Создание объекта класса Net
net = Net()

# Определение функции потерь - перекрёстная энтропия
criterion = nn.CrossEntropyLoss()
# Оптимизация через стохастический градиентный спуск со скоростью обучения 0.01 и моментом 0.9
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)  

# Начало обучения на [epoch_num] количестве эпох
epoch_num = 10
for epoch in range(epoch_num):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # Получение изображений и меток из загрузчика
        inputs, labels = data

        # Сброс градиентов
        optimizer.zero_grad()

        # Прямой проход через нейронную сеть
        outputs = net(inputs)
        # Вычисление потерь между прогнозами
        loss = criterion(outputs, labels)
        # Обратный проход для вычисления градиентов
        loss.backward()
        # Обновление весов на основе вычисленных градиентов
        optimizer.step()

        # Печать статистики
        running_loss += loss.item()
    
        print('[%d, %5d] loss: %.3f' %
            (epoch + 1, i + 1, running_loss / 200))
        running_loss = 0.0

print('Training has finished')

correct = 0  # Число правильных предсказаний
total = 0    # Общее число изображений

# Отключение градиентов на время тестирования
with torch.no_grad():
    # Цикл тестирования по набору данных из тестовой выборки
    for data in test_loader:
        # Получение изображений и меток из загрузчика
        images, labels = data
        # Прямой проход через нейронную сеть
        outputs = net(images)
        # Получение индекса класса с наибольшей вероятностью
        _, predicted = torch.max(outputs.data, 1)
        # Подсчёт общего количества изображений
        total += labels.size(0)
        # Булева функция для подсчёта количества правильных предсказаний
        correct += (predicted == labels).sum().item()

# Расчёт процента правильных предсказаний по отношению к общему количеству изображений
print('Accuracy of the network on the test images: %d %%' % ( 100 * correct / total ))

torch.save(net, 'modelThreeClasses12e005.pth')

[1,     1] loss: 0.006
[1,     2] loss: 0.005
[1,     3] loss: 0.005
[1,     4] loss: 0.005
[1,     5] loss: 0.005
[1,     6] loss: 0.005
[1,     7] loss: 0.005
[1,     8] loss: 0.005
[1,     9] loss: 0.005
[1,    10] loss: 0.004
[1,    11] loss: 0.004
[1,    12] loss: 0.005
[1,    13] loss: 0.004
[1,    14] loss: 0.005
[1,    15] loss: 0.005
[1,    16] loss: 0.004
[1,    17] loss: 0.005
[1,    18] loss: 0.004
[1,    19] loss: 0.004
[1,    20] loss: 0.004
[1,    21] loss: 0.004
[1,    22] loss: 0.004
[1,    23] loss: 0.003
[1,    24] loss: 0.004
[1,    25] loss: 0.004
[1,    26] loss: 0.004
[1,    27] loss: 0.003
[1,    28] loss: 0.003
[1,    29] loss: 0.003
[1,    30] loss: 0.003
[1,    31] loss: 0.004
[1,    32] loss: 0.003
[1,    33] loss: 0.004
[1,    34] loss: 0.003
[1,    35] loss: 0.004
[1,    36] loss: 0.003
[1,    37] loss: 0.003
[1,    38] loss: 0.003
[1,    39] loss: 0.003
[1,    40] loss: 0.003
[1,    41] loss: 0.003
[1,    42] loss: 0.003
[1,    43] loss: 0.003
[1,    44] 